In [ ]:
import numpy as np
from collections import defaultdict

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
%matplotlib inline

import re
import string
import spacy

# misc:
import warnings
warnings.filterwarnings('ignore')
import tqdm as tqdm
import os

#### Path and configuration class

In [ ]:
class Config:
    
    def __init__(self):
        
        # hyperparameters
        
        # paths
        self.train_file_path = '../input/feedback-prize-2021/train.csv'
        self.train_path = '../input/feedback-prize-2021/train'
        self.test_path = '../input/feedback-prize-2021/test'
        self.model_save_path = '.'
        
        # others
        

In [ ]:
conf = Config()

Note: The terms 'discourse type(s)' and 'classe(s)' are used interchangeably.

# Exploration

In [ ]:
train_df = pd.read_csv(conf.train_file_path)

In [ ]:
#pd.set_option('display.max_colwidth', -1)  # to make dataframe take up the whole width of the screen

In [ ]:
train_df.head()

In [ ]:
train_df.predictionstring[0]

In [ ]:
train_df.info()

In [ ]:
# total number of unique text files
len(train_df['id'].unique())

In [ ]:
# sample discourse text
train_df['discourse_text'][0]

In [ ]:
# utility for printing the text
def print_text(txt_id):
    with open(f'{conf.train_path}/{txt_id}.txt', 'r') as fp:
        text = fp.readlines()
    print(''.join(text))

In [ ]:
# corresponding text for above discourse text
print_text('423A1CA112E2')

In [ ]:
# classes
train_df['discourse_type'].unique().tolist()

In [ ]:
classes = ['Lead', 'Position', 'Evidence', 'Claim', 'Counterclaim', 'Rebuttal', 'Concluding Statement']

In [ ]:
discourse_type_num = train_df['discourse_type_num'].unique().tolist()
discourse_type_num[0:5]

<b>Let's see how many times a particular class appeared at maximum in some text(s) with maximum taken across all texts:</b>

In [ ]:
# This function returns a list, where each index in the list correspond to a particular class (as in the list 
# classes declared above). Each entry in the returned list represents the maximum number of times a class has appeared in some
# text(s).

def get_list_of_max(classes, discourse_type_num):
    list_of_max = []

    for _class in classes:
        mx=0
        for _type in discourse_type_num:
            if _type[-1:].isdigit() and _type[-2:].isdigit():
                if _class == _type[:-3]:
                    curr = _type[-2:]
                    mx = max(mx, int(curr))
            else:
                if _class == _type[:-2]:
                    curr = _type[-1:]
                    mx = max(mx, int(curr))
            
        list_of_max.append(mx)
        
    return list_of_max

In [ ]:
# get the maximum frequencies of all classes
list_of_max = get_list_of_max(classes, discourse_type_num)

In [ ]:
plt.figure(figsize = (7,5))

ax = sns.barplot(x=list_of_max, y=classes)
ax.set_xlabel('max frequency')
ax.set_ylabel('classes')
ax.set_title('Maximum number of times a class can be present in a text')

# we can see from below graph that evidence and claim have appeared a maximum of 12 times in some text(s)

<b>Distribution of classes across all rows of the dataframe:</b><br><br>
Note that the train_df also contains multiple rows for the same text, and a particular discourse type, as we know, can occur multiple times (as shown in the column discourse_type_num) in any given text, therefore the count axis in the below plot takes into account all those multiple occurences of a discourse type of the same text.
<br><br>
We see that the evidence and the claim are the dominant discourse types with rebuttal and concluding statement being the rare ones.

In [ ]:
plt.figure(figsize=(9,7))
sns.set_context('paper',font_scale=1.5)
ax = sns.countplot(x='discourse_type', data=train_df)
ax.set_xticklabels(classes, rotation=40, ha='right');

<b>Let's have a look at class distribution when considering only single instances of classes (discourse_type) in any given text.</b>

In [ ]:
dframe = train_df.drop_duplicates(subset = ['id', 'discourse_type'], keep = 'first', inplace = False)

In [ ]:
dframe.head()

In [ ]:
len(dframe)

In [ ]:
plt.figure(figsize=(9,7))
sns.set_context('paper',font_scale=1.5)
ax = sns.countplot(x='discourse_type', data=dframe)
ax.set_xticklabels(classes, rotation=40, ha='right');

<b>Distribution of the length of complete texts:</b>

In [ ]:
df = train_df.copy()

In [ ]:
df['full_text'] = df['discourse_text'].groupby(df['id']).transform(lambda x: ' '.join(x))

In [ ]:
text_length = df['full_text'].drop_duplicates().apply(len)

fig = plt.figure(figsize=(9,7))

ax = sns.distplot(text_length, kde=False, bins=100, color='r')
ax.set_title('Distribution of Text Length')
ax.set_xlabel("Text Length")
ax.set_ylabel("Frequency")

plt.show()

<b>Distribution for word count:</b>

In [ ]:
word_count = df['full_text'].drop_duplicates().apply(lambda x: len(str(x).split()))

fig = plt.figure(figsize=(9,7))

ax = sns.distplot(word_count, kde=False, bins=100, color='r')
ax.set_title('Distribution of Word Count')
ax.set_xlabel("Word Count")
ax.set_ylabel("Frequency")

plt.show()

<b>Relation between discourse types and discourse length:</b>

In [ ]:
df = train_df.copy()

In [ ]:
df['discourse_len'] = df['discourse_text'].apply(lambda x: len(x.split()))

In [ ]:
fig = plt.figure(figsize=(6,5))

ax = fig.add_axes([0,0,1,1])
ax = df.groupby('discourse_type')['discourse_len'].mean().sort_values().plot(kind="barh", color='lightseagreen')
ax.set_title("Average number of words versus Discourse Type", fontsize=14)
ax.set_xlabel("Average number of words", fontsize = 12)
ax.set_ylabel("")

Distribution of discourse types over discourse length:

In [ ]:
labels = ['Evidence', 'Concluding Statement', 'Lead', 'Rebuttal', 'Counterclaim', 'Position', 'Claim']
feature = 'discourse_len'

data = []
for i in range(len(labels)):
    _data = df.loc[df['discourse_type'] == labels[i]][feature]
    data.append(_data)
    
rows, cols = 4, 2
fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize = (15, 27))
for idx, axis in enumerate(axes.reshape(-1)):
    if idx<len(labels):
        ax = sns.distplot(data[idx], ax = axis, color = 'lightseagreen')
        ax.set_title(labels[idx])
    if idx == len(labels):
        axis.axis('off')  # for turning off the axis of the very last plot

Visualization in a single plot:

In [ ]:
plt.figure(figsize = (12, 5))
ax = sns.kdeplot(data[0])
ax = sns.kdeplot(data[1])
ax = sns.kdeplot(data[2])
ax = sns.kdeplot(data[3])
ax = sns.kdeplot(data[4])
ax = sns.kdeplot(data[5])
ax = sns.kdeplot(data[6])
ax.legend(labels)

<b>Relation between discourse types and frequency of unique words in each discourse text:</b> 

In [ ]:
df['unique_words_count'] = train_df['discourse_text'].apply(lambda x: len(set(str(x).split())))

In [ ]:
fig = plt.figure(figsize=(6,5))

ax = fig.add_axes([0,0,1,1])
ax = df.groupby('discourse_type')['unique_words_count'].mean().sort_values().plot(kind="barh", color='mediumturquoise')
ax.set_title("Unique number of words versus Discourse Type", fontsize=14)
ax.set_xlabel("Unique number of words", fontsize = 12)
ax.set_ylabel("")

Distribution of discourse types over count of unique words in discourse texts:

In [ ]:
labels = ['Evidence', 'Concluding Statement', 'Lead', 'Rebuttal', 'Counterclaim', 'Position', 'Claim']
feature = 'unique_words_count'

data = []
for i in range(len(labels)):
    _data = df.loc[df['discourse_type'] == labels[i]][feature]
    data.append(_data)
    
rows, cols = 4, 2
fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize = (15, 27))
for idx, axis in enumerate(axes.reshape(-1)):
    if idx<len(labels):
        ax = sns.distplot(data[idx], ax = axis, color = 'teal')
        ax.set_title(labels[idx])
    if idx == len(labels):  
        axis.axis('off')  # for turning off the axis of the very last plot

Visualization in a single plot:

In [ ]:
plt.figure(figsize = (12, 5))
ax = sns.kdeplot(data[0])
ax = sns.kdeplot(data[1])
ax = sns.kdeplot(data[2])
ax = sns.kdeplot(data[3])
ax = sns.kdeplot(data[4])
ax = sns.kdeplot(data[5])
ax = sns.kdeplot(data[6])
ax.legend(labels)

<b>Relation between discourse types and average word length of discourse texts:</b>

In [ ]:
df['avg_word_length'] = train_df['discourse_text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

In [ ]:
fig = plt.figure(figsize=(6,5))

ax = fig.add_axes([0,0,1,1])
ax = df.groupby('discourse_type')['avg_word_length'].mean().sort_values().plot(kind="barh", color='steelblue')
ax.set_title("Average word length versus Discourse Type", fontsize=14)
ax.set_xlabel("Average word length", fontsize = 12)
ax.set_ylabel("")

Distribution of discourse types over the average word lengths of discourse texts:

In [ ]:
labels = ['Evidence', 'Concluding Statement', 'Lead', 'Rebuttal', 'Counterclaim', 'Position', 'Claim']
feature = 'avg_word_length'

data = []
for i in range(len(labels)):
    _data = df.loc[df['discourse_type'] == labels[i]][feature]
    data.append(_data)
    
rows, cols = 4, 2
fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize = (15, 27))
for idx, axis in enumerate(axes.reshape(-1)):
    if idx<len(labels):
        ax = sns.distplot(data[idx], ax = axis, color = 'steelblue')
        ax.set_title(labels[idx])
    if idx == len(labels):  
        axis.axis('off')  # for turning off the axis of the very last plot

Visulization in a single plot:

In [ ]:
plt.figure(figsize = (12, 5))
ax = sns.kdeplot(data[0])
ax = sns.kdeplot(data[1])
ax = sns.kdeplot(data[2])
ax = sns.kdeplot(data[3])
ax = sns.kdeplot(data[4])
ax = sns.kdeplot(data[5])
ax = sns.kdeplot(data[6])
ax.legend(labels)

<b>Visualizing the start and end positions of discourse texts:</b>

In [ ]:
data_df = train_df.groupby("discourse_type")[['discourse_end', 'discourse_start']].mean().reset_index().sort_values(
    by = 'discourse_start', 
    ascending = False)

data_df.plot(x='discourse_type',
        kind='barh',
        stacked=False,
        title='Average start and end position absolute',
        figsize=(12,4))

plt.show()

<b>Punctuations:</b>

In [ ]:
# creating temporary corpus for analysing punctuations and stopwords:
def temporary_corpus(target = None):
    corpus=[]
    
    if target == None:
        text_data = train_df['discourse_text'].str.split()
    else:
        text_data = train_df[train_df['discourse_type']==target]['discourse_text'].str.split()
        
    for text in text_data:
        for char in text:
            corpus.append(char)
    return corpus

In [ ]:
plt.figure(figsize=(10,5))
corpus=temporary_corpus()

dic=defaultdict(int)
special = string.punctuation
for token in corpus:
    for character in token:
        if character in special:
            dic[character]+=1
        
x,y=zip(*dic.items())
plt.bar(x,y, color='cadetblue')

<b>Stopwords (with reference to spacy's stopwords list):</b>

In [ ]:
import en_core_web_lg
nlp=en_core_web_lg.load()

In [ ]:
corpus = temporary_corpus()

In [ ]:
def get_frequent_stopwords(corpus):
    
    dic=defaultdict(int)
    for word in corpus:
        if word in nlp.Defaults.stop_words:
            dic[word]+=1

    # getting the top 20 most frequent stop words        
    top=sorted(dic.items(), key=lambda x:x[1],reverse=True)[:20]
    
    return top, dic

In [ ]:
top, dic = get_frequent_stopwords(corpus)

In [ ]:
print(f'Total number of stop words in spacy: {len(nlp.Defaults.stop_words)}')
print(f'Total number of stop words in the given text data which are part of the spacy\'s stop words list: {len(dic)}')

In [ ]:
# top 20 most frequent stop words in our corpus:
plt.figure(figsize = (15,4))
x,y=zip(*top)
plt.bar(x,y, color='goldenrod')

Distribution of discourse types over stopword counts:

In [ ]:
def get_count(text):
    count=0
    for ch in text:
        if ch in nlp.Defaults.stop_words:
            count+=1
    return count

In [ ]:
df = train_df.copy()
df['stop_words_count'] = train_df['discourse_text'].apply(lambda x: get_count(x))

In [ ]:
labels = ['Evidence', 'Concluding Statement', 'Lead', 'Rebuttal', 'Counterclaim', 'Position', 'Claim']

data = []
for i in range(len(labels)):
    _data = df.loc[df['discourse_type'] == labels[i]]['stop_words_count']
    data.append(_data)

In [ ]:
plt.figure(figsize = (12, 5))
ax = sns.kdeplot(data[0])
ax = sns.kdeplot(data[1])
ax = sns.kdeplot(data[2])
ax = sns.kdeplot(data[3])
ax = sns.kdeplot(data[4])
ax = sns.kdeplot(data[5])
ax = sns.kdeplot(data[6])
ax.legend(labels)

<b>n-grams analysis:</b>

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
def get_n_grams(n_grams, top_n = 10):
    
    df_words = pd.DataFrame()
    
    for dt in train_df['discourse_type'].unique():
        
        df = train_df.query('discourse_type == @dt')
        texts = df['discourse_text'].tolist()
        vec = CountVectorizer(lowercase = True, stop_words = 'english',\
                              ngram_range=(n_grams, n_grams)).fit(texts)
        bag_of_words = vec.transform(texts)
        sum_words = bag_of_words.sum(axis=0)
        words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
        cvec_df = pd.DataFrame.from_records(words_freq,\
                                            columns= ['words', 'counts']).sort_values(by="counts", ascending=False)
        cvec_df.insert(0, "Discourse_type", dt)
        cvec_df = cvec_df.iloc[:top_n,:]
        df_words = df_words.append(cvec_df)
        
    return df_words

In [ ]:
unigrams = get_n_grams(n_grams = 1, top_n=10)
unigrams.head()

In [ ]:
def plot_ngram(df, type = "unigrams"):
    
    plt.figure(figsize=(15, 12))
    plt.subplots_adjust(hspace=0.5)

    for n, dt in enumerate(df.Discourse_type.unique()):
        ax = plt.subplot(4, 2, n + 1)
        ax.set_title(f"Most used {type} in {dt}")
        data = df.query('Discourse_type == @dt')[['words', 'counts']].set_index("words").sort_values(by = "counts", ascending = True)
        data.plot(ax=ax, kind = 'barh', color = 'steelblue')
        plt.ylabel("")
    plt.tight_layout()
    plt.show()

In [ ]:
# unigrams
plot_ngram(unigrams)

In [ ]:
# bigrams
bigrams = get_n_grams(n_grams = 2, top_n=10)
plot_ngram(bigrams, 'bigrams')

In [ ]:
# trigrams
trigrams = get_n_grams(n_grams = 3, top_n=10)
plot_ngram(trigrams, 'trigrams')

#### Text Visualization

In [ ]:
train_files = os.listdir(conf.train_path)
test_files = os.listdir(conf.test_path)

for file in range(len(train_files)):
    train_files[file] = str(conf.train_path) + "/" +  str(train_files[file])
for file in range(len(test_files)):
    test_files[file] = str(conf.test_path) + "/" +  str(test_files[file])

In [ ]:
train_files[40]

In [ ]:
train_files[40][35:-4]

In [ ]:
r = 20
ents = []
 
for i, row in train_df[train_df['id'] == train_files[r][35:-4]].iterrows():
    ents.append({
                    'start': int(row['discourse_start']),   
                     'end': int(row['discourse_end']), 
                     'label': row['discourse_type']
                })

with open(train_files[r], 'r') as file: 
    data = file.read()

doc2 = {
    "text": data,
    "ents": ents,
}

colors = {'Lead': 'turquoise',
          'Position': '#f9d5de',
          'Claim': '#adcfad',
          'Evidence': 'wheat',
          'Counterclaim': '#bdf2fa',
          'Concluding Statement': '#eea69e',
          'Rebuttal': '#d1f8f4'}

options = {"ents": train_df.discourse_type.unique().tolist(), "colors": colors}
spacy.displacy.render(doc2, style="ent", options=options, manual=True, jupyter=True)

### To be continued..